Pororo Summarization Extract

ROUGE-N에 대한 F1 SCORE 0.32505909	0.24971946	0.29309712

In [1]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [2]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141815 sha256=5582081f1422d780b9ae8f43556a6fdb6354ca953a854e8c622be10e65e847c1
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [4]:
import numpy as np
import pandas as pd

import json
from pandas.io.json import json_normalize

import re
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [5]:
import os

DIR = "/content/drive/MyDrive/DACON_LG/data/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

In [6]:
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [7]:
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [8]:
train['total'] = train.title + ' ' + train.region + ' ' + train.context
test['total'] = test.title + ' ' + test.region + ' ' + test.context

In [9]:
train.head()

,uid,title,region,context,summary,total
1000,1000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의석을 정돈하여 주시기 ...
1001,1001,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의사팀장 수고하셨습니다....
1002,1002,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제2항 ...
1003,1003,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제3항 ...
1004,1004,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록 완주 의석을 정돈...


In [10]:
test.head()

,uid,title,region,context,total
2000,2000,제235회 본회의 제1차(2012.06.21.),음성,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 음성군의회 제235회 ...,제235회 본회의 제1차(2012.06.21.) 음성 의석을 정돈하여 주시기 ...
2001,2001,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제1항, 음성군의회 제235회 제1차 정례회 회기결정의 건을 상정합니다. ...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제1항, 음성군..."
2002,2002,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다. 제235회 제1차 정례회...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제2항, 회의록..."
2003,2003,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니다. 예산결산특별위원회 구성...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제3항, 예산결..."
2004,2004,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제4항, 환경분야 현지확인 특별위원회 구성결의안을 상정합니다. 대표발의하신...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제4항, 환경분..."


In [11]:
testcopy = test['total']

In [12]:
testcopy

2000    제235회    본회의 제1차(2012.06.21.) 음성 의석을 정돈하여 주시기 ...
2001    제235회    본회의 제1차(2012.06.21.) 음성 의사일정 제1항, 음성군...
2002    제235회    본회의 제1차(2012.06.21.) 음성 의사일정 제2항, 회의록...
2003    제235회    본회의 제1차(2012.06.21.) 음성 의사일정 제3항, 예산결...
2004    제235회    본회의 제1차(2012.06.21.) 음성 의사일정 제4항, 환경분...
                              ...                        
2501    제174회    본회의 제2차(2006.10.16.) 음성 의사일정 제1항, 제17...
2502    제174회    본회의 제2차(2006.10.16.) 음성 의사일정 제2항, 농협 ...
2503    제174회    본회의 제2차(2006.10.16.) 음성 의사일정 제3항, 음성군...
2504    제174회    본회의 제2차(2006.10.16.) 음성 의사일정 제4항, 제4기...
2505    제174회    본회의 제2차(2006.10.16.) 음성 의사일정 제5항, 휴회의...
Name: total, Length: 506, dtype: object

In [13]:
!pip install --upgrade pororo

In [14]:
!pip install kollocate

In [15]:
!pip install koparadigm

In [16]:
!pip install g2pk

In [17]:
testcopy_total_list = testcopy.tolist()

In [18]:
testcopy_total_list[0]

'제235회    본회의 제1차(2012.06.21.) 음성 의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 음성군의회 제235회 제1차 정례회 제1차 본회의를 개의하겠습니다. 먼저 의회사무과장으로부터 보고가 있겠습니다. 의회사무과장입니다. 먼저 지난 제234회 임시회 제1차 본회의에서 의결하여 이송한 안건에 대해서 보고를 드리겠습니다. 2012년 5월 29일 제1차 본회의에서 의결하여 이송한 음성군 지방공무원 정원 조례 일부개정조례안은 2012년 6월 15일자로 공포되었습니다. 다음은 제235회 제1차 정례회 제1차 본회의와 관련하여 보고 드리겠습니다. 「지방자치법」제44조 및 「음성군 회기와 그 운영 등에 관한 조례」제4조제1항에 따라 정례회를 개최하고자 6월 10일자로 집회공고를 하였습니다. 6월 14일 이대웅 의원님 외 2분의 의원님으로 환경분야 현지확인 특별위원회 구성 결의안이, 손달섭 의원님 외 2분의 의원님으로부터 제234회 임시회 시 활동한 주요사업 현지확인 결과 보고의 건이 각각 접수되었습니다. 음성군수로부터 2012년 6월 14일자로 2011회계 세입ㆍ세출 결산 승인안이, 6월 19일자로 2011년도 예비비 지출 승인안이 각각 접수되어 오늘 제1차 정례회를 개최하게 되었습니다. 이상으로 보고를 마치겠습니다. 사무과장님 수고 많이 하셨습니다.'

In [21]:
from pororo import Pororo

# 235
summ_ext = Pororo(task="summarization", model="extractive", lang="ko")
ext_result = []
for i in range(0,236):
  ext_result.append(summ_ext(testcopy_total_list[i]))

IndexError: ignored

In [26]:
# 236
ext_result.append(summ_ext(testcopy_total_list[235]))

In [28]:
# 237
for i in range(237,len(testcopy_total_list)):
  ext_result.append(summ_ext(testcopy_total_list[i]))

In [29]:
ext_result[:10]

['성원이 되었으므로 지금부터 음성군의회 제235회 제1차 정례회 제1차 본회의를 개의하겠습니다. 먼저 의회사무과장으로부터 보고가 있겠습니다. 음성군수로부터 2012년 6월 14일자로 2011회계 세입ㆍ세출 결산 승인안이, 6월 19일자로 2011년도 예비비 지출 승인안이 각각 접수되어 오늘 제1차 정례회를 개최하게 되었습니다.',
 '제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제1항, 음성군의회 제235회 제1차 정례회 회기결정의 건을 상정합니다. 음성군의회 제235회 제1차 정례회 회기는 「음성군의회 회기와 그 운영 등에 관한 조례」제4조제1항의 규정과 의원님들께서 사전에 협의하여 주신 대로 2012년 6월 21일부터 6월 28일까지 8일간 하고자 합니다. 그리고 이번 제1차 정례회 주요의사일정은 환경분야 현지확인 특위 구성의 건, 주요사업 현지확인 결과 보고의 건과 조례안 2건, 2011년도 예비비 지출 승인안, 2011회계 세입ㆍ세출결산 승인안, 2012년도 하반기 군정주요업무 계획 보고 등이 있습니다.',
 '제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다. 제235회 제1차 정례회 회의록 서명의원으로는 의원님들이 사전에 양해하여 주신대로 조천희 의원님, 손달섭 의원님을 선출하고자 하는데, 의원 여러분! 이의가 없으신지요? (「없습니다」하는 의원 있음) 이의가 없으므로 조천희 의원님, 손달섭 의원님이 회의록 서명의원으로 선출되었음을 선포합니다. 두 분 의원님께서는 수고하여 주시기 바랍니다.',
 '제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니다. 먼저 예산결산특별위원회 위원은 손수종 의원님, 이한철 의원님, 남궁유 의원님, 조천희 의원님, 손달섭 의원님, 이대웅 의원님, 김순옥 의원님으로 구성하고자 합니다. 그리고 본 특별위원회 운영기간은 6월 25일 하루 동안 운영하고자 하며, 2011년도 예비비 지출 승

In [31]:
submission = pd.read_csv('/content/drive/MyDrive/DACON_LG/data/sample_submission.csv')

In [32]:
submission

,uid,summary
0,id_2000-AGENDA_1,요약문
1,id_2000-AGENDA_2,요약문
2,id_2000-AGENDA_3,요약문
3,id_2000-AGENDA_4,요약문
4,id_2000-AGENDA_5,요약문
...,...,...
501,id_2096-AGENDA_2,요약문
502,id_2096-AGENDA_3,요약문
503,id_2096-AGENDA_4,요약문
504,id_2096-AGENDA_5,요약문


In [33]:
ext_result = pd.Series(ext_result)

In [34]:
submission['summary'] = ext_result

In [35]:
len(submission)

506

In [43]:
submission

0      성원이 되었으므로 지금부터 음성군의회 제235회 제1차 정례회 제1차 본회의를 개의...
1      제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제1항, 음성군의회 ...
2      제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제2항, 회의록 서명...
3      제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제3항, 예산결산특별...
4      제235회 제1차 정례회 제1차 본회의를 맞이하여 음성군 환경분야 현지확인을 발의하...
                             ...                        
501    제174회 본회의 제2차(2006.10.16.) 음성 의사일정 제1항, 제174회 ...
502    제174회 본회의 제2차(2006.10.16.) 음성 의사일정 제2항, 농협 축산물...
503    제174회 본회의 제2차(2006.10.16.) 음성 의사일정 제3항, 음성군 대소...
504    지역보건의료계획은 지역보건법 제3조 및 동법시행령 제5조에 의거 4년마다 수립하도록...
505    0    제174회 임시회 휴회의 건은 농협축산물공판장 이전관련 현지확인 특별위원회...
Name: summary, Length: 506, dtype: object

In [45]:
submission.to_csv('/content/drive/MyDrive/DACON_LG/data/dacon_pororo_extract.csv', index=False)